## Transfer Resting State Eyetracking Files From USB to Computer (.ps1)

This script will keep a log of when it was last synced. A prompt will appear asking for confirmation of the files that you would like to push to the backup device. It is designed so that any files created after the last sync will be copied over (meaning you could rename a file within the same folder and they won't be copied over again). Still make sure to check if the session folders pushed are the ones expected.  

VERSION: `Structural_Eyetrack_Auto_Push`

In [ ]:
$source = "E:\RestingState_DAY1" 
$destination = "C:\Backupdata\RestingState_DAY1" 
$logFile = "C:\Backupdata\RestingState_DAY1\DAY1_USB_Log.txt" 

if (-not (Test-Path -Path $logFile)) {
    New-Item -Path $logFile -ItemType File
    "Log Created: $(Get-Date)" | Out-File -FilePath $logFile
}

$lastSyncTime = (Get-Content -Path $logFile -Tail 1) -replace "Last Sync:", ""
if ([string]::IsNullOrWhiteSpace($lastSyncTime)) {
    $lastSyncTime = [datetime]::MinValue
} else {
    $lastSyncTime = [datetime]::Parse($lastSyncTime)
}

$newData = Get-ChildItem -Path $source -Directory | Where-Object { $_.CreationTime -gt $lastSyncTime }

if ($newData.Count -eq 0) {
    Write-Output "No new session folders since last synced."
} else {
    foreach ($dir in $newData) {
        $dirSizeBytes = (Get-ChildItem -Path $dir.FullName -Recurse -File | Measure-Object -Property Length -Sum).Sum
        $dirSizeMB = [math]::Round($dirSizeBytes / 1MB, 2)
        $creationTime = $dir.CreationTime

        $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($dirSizeMB) MB, Created: $($creationTime)]
         from $source ? [Y/N]"

        if ($userInput -eq "Y") {
            $destPath = Join-Path -Path $destination -ChildPath $dir.Name
            Copy-Item -Path $dir.FullName -Destination $destPath -Recurse
            Write-Output "Copied $($dir.Name) to the computer"
        }
    }
}

"Last Sync: $(Get-Date)" | Out-File -FilePath $logFile -Append
